In [1]:
import numpy as np
import pandas as pd
import unittest

np.random.seed(50)

In [2]:

# Define sizes of initial-population, children and generations 
initial_population_size = 100
num_of_children = 50 # should be even number
num_of_generation = 500

# position to perform Crossover operation on the combination
crossOver=0.5

# Define fitness parameters
num_of_queens = 8
num_of_checks = 7
target_value = num_of_queens * num_of_checks

In [3]:

# Generate initial-population
def generate_init_pop(pop_size):
    initial_population = []
    for i in range(pop_size):
        initial_population.append(np.random.randint(0,8,8))
    return initial_population


## Get attack positions
def get_atk_position(pos, dist):
    atk_position = np.array([pos-dist, pos, pos+dist], dtype="int32")
    return atk_position

## Get relative position of the queen (to calculate distance)
def get_rel_pos(position_array, idx_val):
    rel_pos = np.empty(8)
    for i in range(len(position_array)):
        if idx_val < i:
            rel_pos[i] = (i - idx_val)
        else:
            rel_pos[i] = (idx_val - i)
            
    return rel_pos

# Fitness score 

## Max fitness score is 8 queens * 7 attacks from other queens = 56
def fitness_score(position_array):
    score = 0
    for i in range(len(position_array)):
        rel_pos = get_rel_pos(position_array, i)
        for j in range(len(position_array)):
            if i == j:
                continue
            atk_position = get_atk_position(position_array[i], rel_pos[j])
            if position_array[j] in atk_position:
                is_attack = True
            else:
                is_attack = False
                score += 1
            # print(position_array[j], atk_position, is_attack, score)
    return score

In [4]:
# generate probabilty based on fitness score
def get_probability(fitness_value, target_value = 56):
    probability = np.empty(len(fitness_value))
    for i in range(len(fitness_value)):
        probability[i] = (np.round(fitness_value[i]/target_value, 5))
    return probability

In [5]:
# sort dataframe based on probability
def prob_sort(population_dataframe):
    population_dataframe = (population_dataframe.sort_values(["probability"], ascending=False).reset_index(drop=True))
    return population_dataframe

# store generated info into a dataframe
def get_dataframe(combination_array):
    population = []
    fitness = []

    for i in range(len(combination_array)):
        combination = combination_array[i]
        population.append(combination)
        fitness.append(fitness_score(combination))

    pop_df = pd.DataFrame({"combination":population, "fitness":fitness})
    pop_df["probability"] = get_probability(pop_df["fitness"])
    pop_df = prob_sort(pop_df)

    return pop_df

In [6]:
# function to perform One Point Crossover
def crossover(parent1, parent2, crossOver=0.5):
    # get Crossover length to slice
    crossOver_len = round(int(len(parent1) * crossOver))
    
    # Slice and perform crossover
    child1 = np.concatenate((parent1[:crossOver_len], parent2[crossOver_len:]), axis=0)
    child2 = np.concatenate((parent2[:crossOver_len], parent1[crossOver_len:]), axis=0)
    
    return child1, child2


# function to perform Random Resetting mutation
def mutation(combination):
    random_queen = np.random.randint(8)
    random_position = int(np.random.randint(0,8,1))
    combination[random_queen] = random_position
    return combination

In [7]:
# generate offspring combination based on performing Crossover and Mutation on selected parents
def get_children(parent_array):

    children = []
    mutated_children = []

    for i in range(0,num_of_children,2):
        p1 = parent_array[i]
        p2 = parent_array[i+1]
    
        (c1, c2) = crossover(p1, p2)
    
        children.append(c1)
        children.append(c2)
    
    for i in range(num_of_children):
        mutated_children.append(mutation(children[i]))
    
    return mutated_children

In [8]:
# Function to generate the right sequence
def generate_sequence(initial_population, children, generations, expected_fitness):
    # get initial population fitness

    init_pop_fitness = get_dataframe(initial_population)
    mating_pool = init_pop_fitness[:children]

    print("expected fitness:", expected_fitness, "\n----------------------------------n")

    for i in range(generations):
        # get expected fitness count
        expected_fitness_count = mating_pool[(mating_pool["fitness"]==expected_fitness)]["fitness"].count()
        if expected_fitness_count > 0:
            print("Gen:", (i-1), 
                ", best-combination:", mating_pool["combination"][0], 
                ", fitness:", mating_pool["fitness"][0],
                 ", probability:", mating_pool["probability"][0]
             )
            break
        gen_pop = []
        gen_pop = get_children(mating_pool["combination"])
        gen_pop = np.concatenate((gen_pop, list(mating_pool["combination"])), axis=0)
        mating_pool = get_dataframe(gen_pop)
    
        print("Gen:", i, 
            ", best-combination:", mating_pool["combination"][0], 
            # mating_pool["combination"][0], mating_pool["combination"][1], mating_pool["combination"][2],
            ", fitness:", mating_pool["fitness"][0],
            ", avg-fitness:", np.round(np.mean(mating_pool["fitness"]), 3),
            ", probability:", mating_pool["probability"][0]
            )
    return mating_pool[mating_pool["probability"]==1]["combination"][0]

In [9]:
# Generate initial-population
init_pop = generate_init_pop(initial_population_size)

# display initial-population
init_pop[:5]

# store right sequence
right_sequence = generate_sequence(init_pop, num_of_children, num_of_generation, target_value)

expected fitness: 56 
----------------------------------n
Gen: 0 , best-combination: [6 5 3 1 4 7 5 0] , fitness: 52 , avg-fitness: 42.92 , probability: 0.92857
Gen: 1 , best-combination: [4 2 7 6 1 3 5 0] , fitness: 54 , avg-fitness: 42.427 , probability: 0.96429
Gen: 2 , best-combination: [4 2 7 6 1 3 5 0] , fitness: 54 , avg-fitness: 42.32 , probability: 0.96429
Gen: 3 , best-combination: [4 2 7 6 1 3 5 0] , fitness: 54 , avg-fitness: 42.4 , probability: 0.96429
Gen: 4 , best-combination: [4 2 7 6 1 3 5 0] , fitness: 54 , avg-fitness: 42.6 , probability: 0.96429
Gen: 5 , best-combination: [4 2 7 6 1 3 5 0] , fitness: 54 , avg-fitness: 42.829 , probability: 0.96429
Gen: 6 , best-combination: [4 2 7 6 1 3 5 0] , fitness: 54 , avg-fitness: 42.96 , probability: 0.96429
Gen: 7 , best-combination: [4 2 7 6 1 3 5 0] , fitness: 54 , avg-fitness: 43.089 , probability: 0.96429
Gen: 8 , best-combination: [4 2 7 6 1 3 5 0] , fitness: 54 , avg-fitness: 43.216 , probability: 0.96429
Gen: 9 , best

Gen: 78 , best-combination: [4 1 7 5 2 0 5 3] , fitness: 54 , avg-fitness: 46.91 , probability: 0.96429
Gen: 79 , best-combination: [1 3 7 7 2 0 6 4] , fitness: 54 , avg-fitness: 46.934 , probability: 0.96429
Gen: 80 , best-combination: [1 7 5 0 2 0 6 3] , fitness: 54 , avg-fitness: 46.956 , probability: 0.96429
Gen: 81 , best-combination: [1 3 7 7 2 0 6 4] , fitness: 54 , avg-fitness: 46.978 , probability: 0.96429
Gen: 82 , best-combination: [1 3 7 5 2 0 6 4] , fitness: 54 , avg-fitness: 47.002 , probability: 0.96429
Gen: 83 , best-combination: [1 3 7 5 2 0 6 4] , fitness: 54 , avg-fitness: 47.024 , probability: 0.96429
Gen: 84 , best-combination: [1 7 2 6 2 0 5 3] , fitness: 54 , avg-fitness: 47.046 , probability: 0.96429
Gen: 85 , best-combination: [4 1 7 6 2 0 5 3] , fitness: 54 , avg-fitness: 47.074 , probability: 0.96429
Gen: 86 , best-combination: [4 1 7 6 2 0 5 3] , fitness: 54 , avg-fitness: 47.1 , probability: 0.96429
Gen: 87 , best-combination: [4 1 7 5 2 0 5 3] , fitness: 5

In [10]:
# Display the right sequence
right_sequence_str = " ".join([str(i) for i in right_sequence.tolist()])
print("Sequence :", right_sequence_str)

Sequence : 1 3 5 7 2 0 6 4
